## 加载微调框架PEFT和STF监督微调训练器STFTrainer

In [1]:
# %%capture #不显示安装过程
import sys, torch
import pandas as pd 
print(sys.executable)
print(torch.__version__, torch.version.cuda)
!pip install huggingface-hub


!pip install unsloth
# Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install --upgrade unsloth

[HAMI-core Msg(33648:140596245272384:libvgpu.c:836)]: Initializing.....
[HAMI-core Warn(33648:140596245272384:multiprocess_memory_limit.c:592)]: Kick dead proc 33073
[HAMI-core Warn(33648:140596245272384:multiprocess_memory_limit.c:592)]: Kick dead proc 33183


/usr/bin/python3.12
2.7.1+cu126 12.6
Looking in indexes: http://pip.58dns.org/simple

[notice] A new release of pip is available: 24.1.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip
Looking in indexes: http://pip.58dns.org/simple

[notice] A new release of pip is available: 24.1.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip
Looking in indexes: http://pip.58dns.org/simple

[notice] A new release of pip is available: 24.1.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [2]:
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # More models at https://huggingface.co/unsloth

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 本地模型目录路径
local_model_path = "/code/ysh/finetuning/data/models/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=local_model_path,  # <-- 使用本地路径
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("模型和分词器已从本地路径成功加载。")

[HAMI-core Msg(33648:140596245272384:libvgpu.c:852)]: Initialized


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[HAMI-core Msg(33767:140394665207616:libvgpu.c:836)]: Initializing.....
/usr/local/lib/python3.12/dist-packages/transformers/loss/loss_for_object_detection.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.2)
  from scipy.optimize import linear_sum_assignment
[HAMI-core Msg(33767:140394665207616:libvgpu.c:852)]: Initialized


Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.25 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


[HAMI-core Msg(33648:140596245272384:memory.c:512)]: orig free=17440440320 total=25393692672 limit=24964497408 usage=813827072
[HAMI-core Msg(33648:140596245272384:memory.c:512)]: orig free=17440440320 total=25393692672 limit=24964497408 usage=813827072


模型和分词器已从本地路径成功加载。


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


##   从这里开始准备数据集

In [5]:
from datasets import load_dataset
dataset = load_dataset("parquet",data_files='./data/finetuning_data_train.parquet', split = "train")
print(dataset.column_names)

Generating train split: 0 examples [00:00, ? examples/s]

['instruction', 'input', 'output', 'text']


In [6]:
# # df = pd.read_parquet('./data/alpaca-gpt4/train.parquet')
# df = pd.read_parquet('data/finetuning_data_train.parquet')

In [7]:
##必须只有两列——一个 instruction 列和一个 output 列。

In [8]:
## 构造assistant单轮对话模式
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 1, # Select more to handle longer conversations
)

Merging columns:   0%|          | 0/4832 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/4832 [00:00<?, ? examples/s]

In [9]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': "Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 1, UserProfile: UserID:1 | Gender:F | Age:1 | Occupation:10 | Zip:48067 | RatedMovies:MovieID:595,rating:5,Genres:Animation|Children's|Musical;MovieID:527,rating:5,Genres:Drama|War;MovieID:1545,rating:4,Genres:Drama;MovieID:2398,rating:4,Genres:Drama;MovieID:594,rating:4,Genres:Animation|Children's|Musical;MovieID:1197,rating:3,Genres:Action|Adventure|Comedy|Romance;MovieID:1029,rating:5,Genres:Animation|Children's|Musical;MovieID:2321,rating:3,Genres:Comedy;MovieID:2791,rating:4,Genres:Comedy;MovieID:3114,rating:4,Genres:Animation|Children's|Comedy;MovieID:531,rating:4,Genres:Children's|Drama;MovieID:2918,rating:4,Genres:Comedy

In [10]:
##将数据集转换为 ShareGPT 格式 的标准化函数
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
"""
{
  "conversations": [
    {
      "from": "human",
      "value": "用户的问题或指令"
    },
    {
      "from": "gpt", 
      "value": "模型的回答"
    }
  ]
}

==>

{
  "conversations": [
    {
      "role": "user",       # 标准化角色名
      "content": "内容..."   # 标准化字段名
    },
    {
      "role": "assistant",  # 标准化角色名
      "content": "内容..."   # 标准化字段名
    }
  ]
}
"""

Unsloth: Standardizing formats (num_proc=96):   0%|          | 0/4832 [00:00<?, ? examples/s]

'\n{\n  "conversations": [\n    {\n      "from": "human",\n      "value": "用户的问题或指令"\n    },\n    {\n      "from": "gpt", \n      "value": "模型的回答"\n    }\n  ]\n}\n\n==>\n\n{\n  "conversations": [\n    {\n      "role": "user",       # 标准化角色名\n      "content": "内容..."   # 标准化字段名\n    },\n    {\n      "role": "assistant",  # 标准化角色名\n      "content": "内容..."   # 标准化字段名\n    }\n  ]\n}\n'

In [11]:
dataset[5]

{'conversations': [{'content': "Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 7, UserProfile: UserID:7 | Gender:M | Age:35 | Occupation:1 | Zip:06810 | RatedMovies:MovieID:733,rating:5,Genres:Action|Adventure|Thriller;MovieID:1580,rating:4,Genres:Action|Adventure|Comedy|Sci-Fi;MovieID:2916,rating:5,Genres:Action|Adventure|Sci-Fi|Thriller;MovieID:377,rating:3,Genres:Action|Romance|Thriller;MovieID:3418,rating:3,Genres:Action|Drama;MovieID:1610,rating:5,Genres:Action|Thriller;MovieID:457,rating:5,Genres:Action|Thriller;MovieID:2571,rating:5,Genres:Action|Sci-Fi|Thriller;MovieID:589,rating:5,Genres:Action|Sci-Fi|Thriller;MovieID:110,rating:5,Genres:Action|Drama|War;MovieID:2028,rating:5,Genres:Action|Drama|War;MovieID:1997,ra

In [12]:
##prompt模版

In [13]:
##Alpaca 格式
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [14]:
## Llama-3 的提示格式，必须使用 instruct 而不是 base 
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""

In [15]:
##ChatML 格式
chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""

In [16]:
##最终使用openai模版，一问一答格式
chat_template = """Strictly follow the following requirements to complete the recommendation of candidate items.
### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/4832 [00:00<?, ? examples/s]

In [17]:
dataset[5]

{'conversations': [{'content': "Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 7, UserProfile: UserID:7 | Gender:M | Age:35 | Occupation:1 | Zip:06810 | RatedMovies:MovieID:733,rating:5,Genres:Action|Adventure|Thriller;MovieID:1580,rating:4,Genres:Action|Adventure|Comedy|Sci-Fi;MovieID:2916,rating:5,Genres:Action|Adventure|Sci-Fi|Thriller;MovieID:377,rating:3,Genres:Action|Romance|Thriller;MovieID:3418,rating:3,Genres:Action|Drama;MovieID:1610,rating:5,Genres:Action|Thriller;MovieID:457,rating:5,Genres:Action|Thriller;MovieID:2571,rating:5,Genres:Action|Sci-Fi|Thriller;MovieID:589,rating:5,Genres:Action|Sci-Fi|Thriller;MovieID:110,rating:5,Genres:Action|Drama|War;MovieID:2028,rating:5,Genres:Action|Drama|War;MovieID:1997,ra

In [18]:
# 查看第一个样本
print("Sample 1:")
print(dataset[0]['text'])
print("\n" + "="*50 + "\n")

# # 查看第二个样本
# print("Sample 2:")
# print(dataset[1]['text'])

Sample 1:
<|begin_of_text|>Strictly follow the following requirements to complete the recommendation of candidate items.
### Instruction:
Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 1, UserProfile: UserID:1 | Gender:F | Age:1 | Occupation:10 | Zip:48067 | RatedMovies:MovieID:595,rating:5,Genres:Animation|Children's|Musical;MovieID:527,rating:5,Genres:Drama|War;MovieID:1545,rating:4,Genres:Drama;MovieID:2398,rating:4,Genres:Drama;MovieID:594,rating:4,Genres:Animation|Children's|Musical;MovieID:1197,rating:3,Genres:Action|Adventure|Comedy|Romance;MovieID:1029,rating:5,Genres:Animation|Children's|Musical;MovieID:2321,rating:3,Genres:Comedy;MovieID:2791,rating:4,Genres:Comedy;MovieID:3114,rating:4,Genres:Animation|Children's

In [19]:
###Huggingface TRL 的 SFTTrainer
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

[HAMI-core Msg(34423:140283758295936:multiprocess_memory_limit.c:500)]: Calling exit handler 34423
[HAMI-core Msg(34424:140212636281728:multiprocess_memory_limit.c:500)]: Calling exit handler 34424


Unsloth: Tokenizing ["text"]:   0%|          | 0/4832 [00:00<?, ? examples/s]

In [20]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.25 GB.
7.117 GB of memory reserved.


In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,832 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.280400
2,1.362200
3,1.359100
4,1.263000
5,1.136800
6,1.184500
7,1.227200
8,0.955200
9,1.088700
10,1.034400


In [22]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

347.4899 seconds used for training.
5.79 minutes used for training.
Peak reserved memory = 7.117 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 30.611 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [23]:
## 使用微调结果进行推理 UserID: 195
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {
        "role": "user",
        "content": "Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 195, UserProfile: UserID:195 | Gender:M | Age:25 | Occupation:12 | Zip:10458 | RatedMovies:MovieID:410,rating:2,Genres:Comedy;MovieID:2748,rating:1,Genres:Action|Adventure;MovieID:2792,rating:2,Genres:Comedy;MovieID:2791,rating:3,Genres:Comedy;MovieID:468,rating:3,Genres:Comedy|Romance;MovieID:1456,rating:2,Genres:Comedy;MovieID:1254,rating:4,Genres:Adventure;MovieID:2808,rating:3,Genres:Action|Sci-Fi;MovieID:3746,rating:5,Genres:Drama|War;MovieID:632,rating:5,Genres:War;MovieID:1296,rating:3,Genres:Drama|Romance;MovieID:2587,rating:4,Genres:Comedy;MovieID:1949,rating:5,Genres:Drama;MovieID:3658,rating:4,Genres:Sci-Fi;MovieID:1090,rating:5,Genres:Drama|War;MovieID:3686,rating:2,Genres:Thriller;MovieID:950,rating:5,Genres:Mystery;MovieID:2373,rating:2,Genres:Action|Adventure;MovieID:2297,rating:3,Genres:Drama|Romance;MovieID:2296,rating:1,Genres:Comedy;MovieID:2174,rating:3,Genres:Comedy|Fantasy;MovieID:900,rating:3,Genres:Musical|Romance;MovieID:1200,rating:3,Genres:Action|Sci-Fi|Thriller|War;MovieID:1884,rating:4,Genres:Comedy|Drama;MovieID:3593,rating:2,Genres:Action|Sci-Fi;MovieID:2488,rating:4,Genres:Drama|Horror|Thriller;MovieID:942,rating:5,Genres:Crime|Film-Noir|Mystery;MovieID:173,rating:1,Genres:Action|Adventure|Sci-Fi;MovieID:3504,rating:5,Genres:Comedy|Drama;MovieID:2113,rating:2,Genres:Horror|Thriller;MovieID:3062,rating:4,Genres:Action|Drama|War;MovieID:1953,rating:4,Genres:Action|Crime|Drama|Thriller;MovieID:3090,rating:4,Genres:Drama;MovieID:3088,rating:5,Genres:Comedy;MovieID:3095,rating:5,Genres:Drama;MovieID:910,rating:4,Genres:Comedy|Crime;MovieID:2004,rating:2,Genres:Comedy|Horror;MovieID:3471,rating:3,Genres:Drama|Sci-Fi;MovieID:2340,rating:3,Genres:Romance;MovieID:3951,rating:5,Genres:Drama;MovieID:3134,rating:4,Genres:Drama|War;MovieID:3327,rating:4,Genres:Documentary;MovieID:3092,rating:5,Genres:Drama;MovieID:2997,rating:4,Genres:Comedy;MovieID:2630,rating:4,Genres:Drama;MovieID:2908,rating:4,Genres:Drama;MovieID:3386,rating:5,Genres:Drama|Mystery;MovieID:2512,rating:4,Genres:Drama;MovieID:2064,rating:5,Genres:Comedy|Documentary;MovieID:2938,rating:5,Genres:Drama;MovieID:6,rating:2,Genres:Action|Crime|Thriller;MovieID:1361,rating:5,Genres:Documentary;MovieID:1796,rating:4,Genres:Action|Drama;MovieID:3730,rating:4,Genres:Drama|Mystery;MovieID:2544,rating:3,Genres:Drama;MovieID:2585,rating:4,Genres:Drama|Romance;MovieID:392,rating:5,Genres:Adventure|Children's;MovieID:1237,rating:5,Genres:Drama;MovieID:3192,rating:4,Genres:Drama;MovieID:1230,rating:4,Genres:Comedy|Romance;MovieID:2560,rating:4,Genres:Drama|Horror;MovieID:126,rating:2,Genres:Adventure|Children's|Fantasy;MovieID:330,rating:4,Genres:Comedy|Horror;MovieID:2860,rating:3,Genres:Comedy;MovieID:3532,rating:4,Genres:Comedy;MovieID:849,rating:2,Genres:Action|Adventure|Sci-Fi|Thriller;MovieID:1248,rating:5,Genres:Crime|Film-Noir|Thriller;MovieID:441,rating:4,Genres:Comedy;MovieID:3700,rating:4,Genres:Drama|Sci-Fi;MovieID:2617,rating:3,Genres:Action|Adventure|Horror|Thriller;MovieID:904,rating:5,Genres:Mystery|Thriller;MovieID:1250,rating:4,Genres:Drama|War;MovieID:2402,rating:1,Genres:Action|War;MovieID:611,rating:2,Genres:Action|Horror|Sci-Fi;MovieID:3683,rating:4,Genres:Drama|Film-Noir;MovieID:3245,rating:5,Genres:Drama;MovieID:3429,rating:4,Genres:Animation|Comedy;MovieID:1859,rating:4,Genres:Drama;MovieID:2624,rating:4,Genres:Drama;MovieID:1258,rating:4,Genres:Horror;MovieID:80,rating:4,Genres:Drama;MovieID:908,rating:4,Genres:Drama|Thriller;MovieID:349,rating:4,Genres:Action|Adventure|Thriller;MovieID:2966,rating:4,Genres:Drama;MovieID:1464,rating:5,Genres:Mystery;MovieID:3456,rating:5,Genres:Drama;MovieID:670,rating:5,Genres:Drama;MovieID:1300,rating:4,Genres:Drama;MovieID:2010,rating:5,Genres:Sci-Fi;MovieID:1217,rating:5,Genres:Drama|War;MovieID:750,rating:5,Genres:Sci-Fi|War;MovieID:1216,rating:5,Genres:Adventure|Romance;MovieID:2688,rating:4,Genres:Drama|Thriller;MovieID:3022,rating:5,Genres:Comedy;MovieID:1212,rating:4,Genres:Mystery|Thriller;MovieID:1931,rating:4,Genres:Adventure;MovieID:2905,rating:4,Genres:Action|Adventure;MovieID:3030,rating:4,Genres:Comedy|Drama|Western;MovieID:1348,rating:5,Genres:Horror;MovieID:669,rating:5,Genres:Drama,\n CandidateItems: 3944:Comedy|Drama;3563:Action|Horror;2517:Horror;2607:Drama;551:Children's|Comedy|Musical;3764:Action|Crime|Thriller;2219:Mystery|Thriller;1356:Action|Adventure|Sci-Fi;688:Action|Adventure|Comedy|War;1702:Children's|Comedy|Fantasy;2127:Drama|Romance;1677:Comedy;2347:Action;237:Comedy|Romance;1047:Action|Thriller;3093:Drama|Western;2660:Sci-Fi;3392:Comedy;2206:Mystery|Thriller;3740:Action|Comedy;2064:Comedy|Documentary;2545:Comedy;3721:Comedy;2644:Horror;3778:Comedy|Drama;2247:Comedy;2807:Action|Sci-Fi;2632:Drama;1301:Sci-Fi;3156:Comedy|Drama|Sci-Fi;464:Action|Adventure|Crime|Thriller;329:Action|Adventure|Sci-Fi;2253:Action|Comedy|Fantasy;2490:Action|Thriller;1556:Action|Romance|Thriller;821:Romance;2788:Comedy;634:Comedy;3007:Documentary;506:Drama;1871:Comedy|Drama;1375:Action|Adventure|Sci-Fi;1183:Drama|Romance|War;2339:Comedy|Romance;1473:Action|Comedy|Crime|Drama;2831:Drama;1374:Action|Adventure|Sci-Fi;2770:Comedy;169:Adventure|Children's|Drama;3436:Drama|Romance.\n"
        
    },
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids, 
    streamer = text_streamer, 
    max_new_tokens = 128, 
    pad_token_id = tokenizer.eos_token_id,
    temperature=0.01,  # ← 添加温度控制
    do_sample=True,   # ← 必须开启采样才能让 temperature 起作用！
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Recommended movie ID: [2219, 3764, 1375, 1374, 464, 329, 2490, 1556, 2339, 1473]<|end_of_text|>


In [ ]:
3944:Comedy|Drama;
3563:Action|Horror;
2517:Horror;
2607:Drama;
551:Children's|Comedy|Musical;
3764:Action|Crime|Thriller;
2219:Mystery|Thriller;
1356:Action|Adventure|Sci-Fi;
688:Action|Adventure|Comedy|War;
1702:Children's|Comedy|Fantasy;
2127:Drama|Romance;
1677:Comedy;
2347:Action;
237:Comedy|Romance;
1047:Action|Thriller;
3093:Drama|Western;
2660:Sci-Fi;
3392:Comedy;
2206:Mystery|Thriller;
3740:Action|Comedy;
2064:Comedy|Documentary;
2545:Comedy;
3721:Comedy;
2644:Horror;
3778:Comedy|Drama;
2247:Comedy;
2807:Action|Sci-Fi;
2632:Drama;
1301:Sci-Fi;
3156:Comedy|Drama|Sci-Fi;
464:Action|Adventure|Crime|Thriller;
329:Action|Adventure|Sci-Fi;
2253:Action|Comedy|Fantasy;
2490:Action|Thriller;
1556:Action|Romance|Thriller;
821:Romance;
2788:Comedy;
634:Comedy;
3007:Documentary;
506:Drama;
1871:Comedy|Drama;
1375:Action|Adventure|Sci-Fi;
1183:Drama|Romance|War;
2339:Comedy|Romance;
1473:Action|Comedy|Crime|Drama;
2831:Drama;
1374:Action|Adventure|Sci-Fi;
2770:Comedy;
169:Adventure|Children's|Drama;3436:Drama|Romance.\n"

In [24]:
###UserID: 195 'Recommended movie ID: [3007, 1183, 1702, 3764, 1301, 2247, 1356, 329, 1374, 1375]'

## 保存、加载微调模型

In [25]:
# 这仅保存了 LoRA 适配器，并未保存整个模型。要保存为 16 位或 GGUF，请用下面的
model.save_pretrained("outputs/lora_model") # Local saving
tokenizer.save_pretrained("outputs/lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('outputs/lora_model/tokenizer_config.json',
 'outputs/lora_model/special_tokens_map.json',
 'outputs/lora_model/chat_template.jinja',
 'outputs/lora_model/tokenizer.json')

In [26]:
##加载训练好的lora块进行推理

In [27]:
##加载模型，使用微调结果进行推理 UID：618
##如果想加载刚刚保存的 LoRA 适配器以进行推理，请将 False 更改为 True
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "outputs/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass

messages = [                    
    # Change below!
    {"role": "user",
     "content": "Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 618, UserProfile: UserID:618 | Gender:M | Age:25 | Occupation:0 | Zip:74105 | RatedMovies:MovieID:1971,rating:3,Genres:Horror;MovieID:3596,rating:1,Genres:Comedy;MovieID:490,rating:2,Genres:Thriller;MovieID:2792,rating:2,Genres:Comedy;MovieID:3755,rating:3,Genres:Action|Adventure|Thriller;MovieID:648,rating:2,Genres:Action|Adventure|Mystery;MovieID:3513,rating:3,Genres:Drama|Thriller;MovieID:3409,rating:3,Genres:Drama|Thriller;MovieID:70,rating:4,Genres:Action|Comedy|Crime|Horror|Thriller;MovieID:2454,rating:3,Genres:Horror|Sci-Fi;MovieID:3947,rating:3,Genres:Thriller;MovieID:3918,rating:2,Genres:Horror;MovieID:1965,rating:2,Genres:Comedy|Sci-Fi;MovieID:3917,rating:4,Genres:Horror;MovieID:3930,rating:2,Genres:Horror;MovieID:3624,rating:4,Genres:Action;MovieID:1171,rating:4,Genres:Comedy;MovieID:2291,rating:3,Genres:Drama|Romance;MovieID:2791,rating:4,Genres:Comedy;MovieID:1673,rating:3,Genres:Drama;MovieID:3481,rating:4,Genres:Comedy;MovieID:551,rating:2,Genres:Children's|Comedy|Musical;MovieID:3578,rating:4,Genres:Action|Drama;MovieID:589,rating:4,Genres:Action|Sci-Fi|Thriller;MovieID:1249,rating:2,Genres:Thriller;MovieID:2858,rating:5,Genres:Comedy|Drama;MovieID:1259,rating:4,Genres:Adventure|Comedy|Drama;MovieID:1250,rating:5,Genres:Drama|War;MovieID:110,rating:4,Genres:Action|Drama|War;MovieID:3347,rating:4,Genres:Drama;MovieID:2717,rating:3,Genres:Comedy|Horror;MovieID:1196,rating:4,Genres:Action|Adventure|Drama|Sci-Fi|War;MovieID:1130,rating:2,Genres:Horror;MovieID:2144,rating:3,Genres:Comedy;MovieID:2746,rating:3,Genres:Comedy|Horror|Musical,\n CandidateItems: 1969:Horror;1569:Comedy|Romance;639:Comedy;2624:Drama;1904:Comedy|Drama;1976:Horror;3508:Western;611:Action|Horror|Sci-Fi;3698:Action|Adventure|Sci-Fi;348:Comedy;3684:Drama|Romance;175:Drama;3587:Horror;1273:Comedy|Drama;2553:Horror|Sci-Fi|Thriller;2454:Horror|Sci-Fi;1928:Western;3179:Drama;1972:Horror;467:Comedy;1973:Horror;208:Action|Adventure;1264:Action|Drama|Mystery|Romance|Thriller;1197:Action|Adventure|Comedy|Romance;1970:Horror;3579:Drama;3537:Comedy|Drama;296:Crime|Drama;3008:Thriller;497:Comedy|Romance;1793:Comedy;321:Drama;309:Drama;2128:Comedy;3730:Drama|Mystery;865:Drama;2450:Adventure|Children's|Sci-Fi;2150:Comedy;891:Horror|Thriller;1986:Horror;933:Comedy|Romance|Thriller;1977:Horror;2456:Horror|Sci-Fi;696:Thriller;2416:Comedy;2241:Comedy;254:Drama;2283:Drama;3810:Drama|Thriller;1812:Children's|Comedy|Drama.\n"
     
    },
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

##控温
_ = model.generate(
    input_ids,
    streamer=text_streamer,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.01,  # ← 添加温度控制
    do_sample=True,   # ← 必须开启采样才能让 temperature 起作用！
)

==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.25 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


[HAMI-core Msg(33648:140596245272384:memory.c:512)]: orig free=10140254208 total=25393692672 limit=24964497408 usage=830868480
[HAMI-core Msg(33648:140596245272384:memory.c:512)]: orig free=10140254208 total=25393692672 limit=24964497408 usage=830868480


Recommended movie ID: [1973, 1970, 1976, 1972, 2454, 2456, 2450, 3579, 3587, 1986]<|end_of_text|>


In [28]:
## 618'Recommended movie ID: [2456, 1976, 1969, 3579, 1972, 1977, 1986, 611, 2624]'